In [1]:
!git clone https://github.com/Cld338/Aspect-Term-Extraction-and-Analysis-Mecab ATE
!pip install transformers
!python -m pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

fatal: destination path 'ATE' already exists and is not an empty directory.
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-9ukyueeb
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-9ukyueeb
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Using cached boto3-1.15.18-py2.py3-none-any.whl (129 kB)
  Using cached gluonnlp-0.10.0.tar.gz (344 kB)
  Preparing metadata (setup.py) ... done
  Using cached mxnet-1.7.0.post2-py2.py3-none-manylinux2014_x86_64.whl (54.7 MB)
INFO: pip is looking at multiple versions of kobert to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement onnxruntime<=1.8.0,==1.8.0 (from kobert) (from versions: 1.12.0, 1.12.1, 1.13.1, 1.14.0, 1.14.1, 1.15.0, 1.1

In [2]:
!pip install jsonlines

In [3]:
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 229 kB in 2s (116 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
openjdk-8-jdk is already the newe

In [4]:
!pip install mecab-python3

# import

In [5]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [6]:
from ATE.model.bert import bert_ATE, bert_ABSA
from ATE.data.dataset import dataset_ATM, dataset_ABSA

In [7]:
from torch.utils.data import DataLoader, ConcatDataset
from transformers import BertTokenizer
import torch
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import time
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [8]:
epoch = 15
ATE_file = f"bert_ATE_epoch{epoch}.pkl"
ABSA_file = f"bert_ABSA_epoch{epoch}.pkl"
file_save_path = "/content/gdrive/MyDrive/2023_sw_inje_proj/bert-base-multilingual-uncased"

# Settings

In [9]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pretrain_model_name = "bert-base-multilingual-uncased"
tokenizer = BertTokenizer.from_pretrained(pretrain_model_name)
lr = 2e-5
model_ATE = bert_ATE(pretrain_model_name).to(DEVICE)
optimizer_ATE = torch.optim.Adam(model_ATE.parameters(), lr=lr)
model_ABSA = bert_ABSA(pretrain_model_name).to(DEVICE)
optimizer_ABSA = torch.optim.Adam(model_ABSA.parameters(), lr=lr)

In [10]:
from konlpy.tag import Mecab
mecab = Mecab()

In [11]:


def evl_time(t):
    min, sec= divmod(t, 60)
    hr, min = divmod(min, 60)
    return int(hr), int(min), int(sec)

def load_model(model, path):
    model.load_state_dict(torch.load(path), strict=False)
    return model

def save_model(model, name):
    torch.save(model.state_dict(), name)

# Acpect Term Extraction

In [51]:
# laptops_train_ds = dataset_ATM(pd.read_csv("ATE/data/laptops_train.csv"), tokenizer)
# laptops_test_ds = dataset_ATM(pd.read_csv("ATE/data/laptops_test.csv"), tokenizer)
# restaurants_train_ds = dataset_ATM(pd.read_csv("ATE/data/restaurants_train.csv"), tokenizer)
# restaurants_test_ds = dataset_ATM(pd.read_csv("ATE/data/restaurants_test.csv"), tokenizer)
# twitter_train_ds = dataset_ATM(pd.read_csv("ATE/data/twitter_train.csv"), tokenizer)
# twitter_test_ds = dataset_ATM(pd.read_csv("ATE/data/twitter_test.csv"), tokenizer)
data_path = "/content/gdrive/MyDrive/2023_sw_inje_proj/data"
custom_train_ds = dataset_ATM(pd.read_csv(f"{data_path}/processed_nikluge-sa-2022-train.csv"), mecab)
custom_test_ds = dataset_ATM(pd.read_csv(f"{data_path}/processed_nikluge-sa-2022-dev.csv"), mecab)

In [52]:
# w,x,y,z = laptops_train_ds.__getitem__(121)
# print(w)
# print(x)
# print(x.size())
# print(y)
# print(y.size())
# print(z)
# print(z.size())

In [53]:

# train_ds = ConcatDataset([laptops_train_ds, restaurants_train_ds, twitter_train_ds])
# test_ds = ConcatDataset([laptops_test_ds, restaurants_test_ds, twitter_test_ds])
train_ds = ConcatDataset([custom_train_ds])
test_ds = ConcatDataset([custom_train_ds])

In [54]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True)

    tags_tensors = [s[2] for s in samples]
    tags_tensors = pad_sequence(tags_tensors, batch_first=True)

    pols_tensors = [s[3] for s in samples]
    pols_tensors = pad_sequence(pols_tensors, batch_first=True)

    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)

    return ids_tensors, tags_tensors, pols_tensors, masks_tensors

In [55]:
train_loader = DataLoader(train_ds, batch_size=5, collate_fn=create_mini_batch, shuffle = True)
test_loader = DataLoader(test_ds, batch_size=50, collate_fn=create_mini_batch, shuffle = True)

In [56]:
# for batch in train_loader:
#     w,x,y,z = batch
#     print(w)
#     print(w.size())
#     print(x)
#     print(x.size())
#     print(y)
#     print(y.size())
#     print(z)
#     print(z.size())
#     break

In [57]:
def train_model_ATE(loader, epochs):
    all_data = len(loader)
    for epoch in range(epochs):
        finish_data = 0
        losses = []
        current_times = []
        correct_predictions = 0

        for data in loader:
            t0 = time.time()
            ids_tensors, tags_tensors, _,  masks_tensors = data
            ids_tensors = ids_tensors.to(DEVICE)
            tags_tensors = tags_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            loss = model_ATE(ids_tensors=ids_tensors, tags_tensors=tags_tensors, masks_tensors=masks_tensors)
            losses.append(loss.item())
            loss.backward()
            optimizer_ATE.step()
            optimizer_ATE.zero_grad()

            finish_data += 1
            current_times.append(round(time.time()-t0,3))
            current = np.mean(current_times)
            hr, min, sec = evl_time(current*(all_data-finish_data) + current*all_data*(epochs-epoch-1))
            print('epoch:', epoch, " batch:", finish_data, "/" , all_data, " loss:", np.mean(losses), " hr:", hr, " min:", min," sec:", sec)

        save_model(model_ATE, f'{file_save_path}/{ATE_file}')

def test_model_ATE(loader):
    pred = []
    trueth = []
    with torch.no_grad():
        for data in loader:

            ids_tensors, tags_tensors, _, masks_tensors = data
            ids_tensors = ids_tensors.to(DEVICE)
            tags_tensors = tags_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            outputs = model_ATE(ids_tensors=ids_tensors, tags_tensors=None, masks_tensors=masks_tensors)

            _, predictions = torch.max(outputs, dim=2)

            pred += list([int(j) for i in predictions for j in i ])
            trueth += list([int(j) for i in tags_tensors for j in i ])

    return trueth, pred



In [58]:
# %time train_model_ATE(train_loader, epoch)

In [59]:
model_ATE = load_model(model_ATE, f'{file_save_path}/{ATE_file}')

In [60]:
%time x, y = test_model_ATE(test_loader)
print(classification_report(x, y, target_names=[str(i) for i in range(3)]))
with open(f"{file_save_path}/ATE_epoch{epoch}_report.txt", "w") as file:
  file.write(classification_report(x, y, target_names=[str(i) for i in range(3)]))

AttributeError: ignored

              precision    recall  f1-score   support

           0       0.64      0.74      0.69       497
           1       0.73      0.59      0.65       710
           2       0.83      0.87      0.85      1239

    accuracy                           0.76      2446
   macro avg       0.73      0.73      0.73      2446
weighted avg       0.76      0.76      0.76      2446



# Aspect Based Sentiment Analysis

In [61]:
laptops_train_ds = dataset_ABSA(pd.read_csv("ATE/data/laptops_train.csv"), tokenizer)
laptops_test_ds = dataset_ABSA(pd.read_csv("ATE/data/laptops_test.csv"), tokenizer)
restaurants_train_ds = dataset_ABSA(pd.read_csv("ATE/data/restaurants_train.csv"), tokenizer)
restaurants_test_ds = dataset_ABSA(pd.read_csv("ATE/data/restaurants_test.csv"), tokenizer)
twitter_train_ds = dataset_ABSA(pd.read_csv("ATE/data/twitter_train.csv"), tokenizer)
twitter_test_ds = dataset_ABSA(pd.read_csv("ATE/data/twitter_test.csv"), tokenizer)

In [62]:
w,x,y,z = laptops_train_ds.__getitem__(121)
print(w)
print(len(w))
print(x)
print(len(x))
print(y)
print(len(y))
print(z)

['[cls]', 'the', 'battery', 'life', 'seems', 'to', 'be', 'very', 'good', ',', 'and', 'have', 'had', 'no', 'issues', 'with', 'it', '.', '[sep]', 'battery', 'life']
21
tensor([  100, 10103, 34794, 10287, 32681, 10114, 10346, 12495, 12050,   117,
        10110, 10574, 10407, 10181, 17156, 10171, 10197,   119,   100, 34794,
        10287])
21
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1])
21
tensor(2)


In [63]:
def create_mini_batch2(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True)

    segments_tensors = [s[2] for s in samples]
    segments_tensors = pad_sequence(segments_tensors, batch_first=True)

    label_ids = torch.stack([s[3] for s in samples])

    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)

    return ids_tensors, segments_tensors, masks_tensors, label_ids

In [64]:
train_ds = ConcatDataset([laptops_train_ds, restaurants_train_ds, twitter_train_ds])
test_ds = ConcatDataset([laptops_test_ds, restaurants_test_ds, twitter_test_ds])

train_loader = DataLoader(train_ds, batch_size=4, collate_fn=create_mini_batch2, shuffle = True)
test_loader = DataLoader(test_ds, batch_size=50, collate_fn=create_mini_batch2, shuffle = True)

In [65]:
# for batch in train_loader:
#     w,x,y,z = batch
#     print(w)
#     print(w.size())
#     print(x)
#     print(x.size())
#     print(y)
#     print(y.size())
#     print(z)
#     print(z.size())
#     break

In [66]:
def train_model_ABSA(loader, epochs):
    all_data = len(loader)
    for epoch in range(epochs):
        finish_data = 0
        losses = []
        current_times = []
        correct_predictions = 0

        for data in loader:
            t0 = time.time()
            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            label_ids = label_ids.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            loss = model_ABSA(ids_tensors=ids_tensors, lable_tensors=label_ids, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            losses.append(loss.item())
            loss.backward()
            optimizer_ABSA.step()
            optimizer_ABSA.zero_grad()

            finish_data += 1
            current_times.append(round(time.time()-t0,3))
            current = np.mean(current_times)
            hr, min, sec = evl_time(current*(all_data-finish_data) + current*all_data*(epochs-epoch-1))
            print('epoch:', epoch, " batch:", finish_data, "/" , all_data, " loss:", np.mean(losses), " hr:", hr, " min:", min," sec:", sec)

        save_model(model_ABSA, f'{file_save_path}/{ABSA_file}')

def test_model_ABSA(loader):
    pred = []
    trueth = []
    with torch.no_grad():
        for data in loader:

            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            outputs = model_ABSA(ids_tensors, None, masks_tensors=masks_tensors, segments_tensors=segments_tensors)

            _, predictions = torch.max(outputs, dim=1)

            pred += list([int(i) for i in predictions])
            trueth += list([int(i) for i in label_ids])

    return trueth, pred



In [67]:
for data in train_loader:
  print(data)
  break

(tensor([[  100,   143, 10981, 15640, 52245, 10103, 34794, 10359, 10127, 10816,
         28032, 23578, 10146, 13594,   143, 31789, 10287, 10108, 10170, 18030,
         10502, 10704, 26344, 36555, 15503, 10902,   128, 18030, 10108, 11416,
           119,   100, 34794,     0,     0,     0],
        [  100, 10387, 10159,   119,   119, 10578, 10346, 11746, 14666,   119,
           119, 39643, 10935,   107,   107, 11811, 46646, 39199, 59299,   143,
         88764, 10167,   107,   107, 86229, 12574,   106,   100, 46646, 39199,
             0,     0,     0,     0,     0,     0],
        [  100, 11162,   118, 10387, 59571, 10107, 17826, 10171,   143, 36186,
           131, 10139, 15785, 35403, 14086,   117, 12317, 10127, 12125,   143,
         18067, 10131, 10986, 36186, 57846,   119, 40978, 10387, 12134,   144,
           119,   119,   100, 40978, 10387, 12134],
        [  100, 25044, 10114, 22611, 10103, 67176, 47297, 10108, 16674, 10572,
         10142,   117, 13446, 12447, 10110, 18914, 24

In [68]:
# %time train_model_ABSA(train_loader, epoch)

In [69]:
model_ABSA = load_model(model_ABSA, f'{file_save_path}/{ABSA_file}')

In [70]:
%time x, y = test_model_ABSA(test_loader)
print(classification_report(x, y, target_names=[str(i) for i in range(3)]))
with open(f"{file_save_path}/ABSA_epoch{epoch}_report.txt", "w") as file:
  file.write(classification_report(x, y, target_names=[str(i) for i in range(3)]))

CPU times: user 5.5 s, sys: 48.6 ms, total: 5.55 s
Wall time: 5.51 s
              precision    recall  f1-score   support

           0       0.64      0.74      0.69       497
           1       0.73      0.59      0.65       710
           2       0.83      0.87      0.85      1239

    accuracy                           0.76      2446
   macro avg       0.73      0.73      0.73      2446
weighted avg       0.76      0.76      0.76      2446



# Morphs

# ATE + ABSA

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df=0, max_features=512)
tfidf_vectorizer = tfidf.fit([" ".join(i[0]) for i in train_ds])

In [72]:
def predict_model_ABSA(sentence, aspect, tokenizer):
    # t1 = tokenizer.tokenize(sentence)
    # t2 = tokenizer.tokenize(aspect)
    t1 = tokenizer.morphs(sentence)
    t2 = tokenizer.morphs(aspect)

    word_pieces = ['[CLS]']
    word_pieces += t1
    word_pieces += ['[SEP]']
    word_pieces += t2

    segment_tensor = [0] + [0]*len(t1) + [0] + [1]*len(t2)

    ids = tokenizer.convert_tokens_to_ids(word_pieces)

    input_tensor = torch.tensor([ids]).to(DEVICE)
    segment_tensor = torch.tensor(segment_tensor).to(DEVICE)

    with torch.no_grad():
        outputs = model_ABSA(input_tensor, None, None, segments_tensors=segment_tensor)
        _, predictions = torch.max(outputs, dim=1)

    return word_pieces, predictions, outputs

def predict_model_ATE(sentence, tokenizer):
    word_pieces = []
    # tokens = tokenizer.tokenize(sentence)
    tokens = tokenizer.morphs(sentence)
    word_pieces += tokens

    # ids = tokenizer.convert_tokens_to_ids(word_pieces)
    ids = tfidf_vectorizer.transform([sentence]).toarray()
    #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    #@@@@@@@@@@@@@@@@@@@@@@@@@여기 안됨@@@@@@@@@@@@@@@@@@@@@@@@@
    #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    # ids array size를 512fh 변경할 것
    input_tensor = torch.LongTensor(ids).to(DEVICE)

    with torch.no_grad():
        outputs = model_ATE(input_tensor, None, None)
        _, predictions = torch.max(outputs, dim=2)
    predictions = predictions[0].tolist()

    return word_pieces, predictions, outputs

def ATE_ABSA(text):
    terms = []
    word = ""
    x, y, z = predict_model_ATE(text, mecab)
    for i in range(len(y)):
        if y[i] == 1:
            if len(word) != 0:
                terms.append(word.replace("##","").replace(" ", ""))
            word = x[i]
        if y[i] == 2:
            word += (" " + x[i])


    if len(word) != 0:
            terms.append(word.replace("##",""))

    print("tokens:", x)
    print("ATE:", terms)

    # if len(terms) != 0:
        # for i in terms:
            # _, c, p = predict_model_ABSA(text, i, tokenizer)
            # print("term:", [i], "class:", [int(c)], "ABSA:", [float(p[0][0]), float(p[0][1]), float(p[0][2])])

In [73]:
model_ABSA = load_model(model_ABSA, f'{file_save_path}/{ABSA_file}')
model_ATE = load_model(model_ATE, f'{file_save_path}/{ATE_file}')

In [74]:
import re

In [75]:
text = "깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면 금방 우러난다."
ATE_ABSA(text)

tokens: ['깔끔', '하', '게', '부직포', '포장', '으로', '되', '어', '있', '어서', '그냥', '뜨거운', '물', '에', '풍덩', '넣', '어', '놓', '고', '좀', '휘젓', '어', '주', '면', '금방', '우러난다', '.']
ATE: []


In [76]:
ATE_ABSA("제품만족도 94% 제품이라 믿고 사용해본 #산다화클렌징오일")

tokens: ['제품', '만족', '도', '94', '%', '제품', '이', '라', '믿', '고', '사용', '해', '본', '#', '산다화', '클렌징', '오일']
ATE: []


In [77]:
text = "목욕할 때마다 넣어봤는데(샤워는 자주 해도 목욕은 그렇게 자주가 아님.. 이것도 약재는 약재이므로 용법은 알아서;;)신선한 한약풀 냄새가 욕실에 퍼져서 기분이 좋아졌다."

ATE_ABSA(text)

tokens: ['목욕', '할', '때', '마다', '넣', '어', '봤', '는데', '(', '샤워', '는', '자주', '해도', '목욕', '은', '그렇게', '자주', '가', '아님', '.', '.', '이것', '도', '약재', '는', '약재', '이', '므로', '용법', '은', '알', '아서', ';;)', '신선', '한', '한', '약풀', '냄새', '가', '욕실', '에', '퍼져서', '기분', '이', '좋', '아', '졌', '다', '.']
ATE: []


In [78]:
text = "고밀도 폼 소재라..빈틈이 없고 커버만 관리하면 되고, 항균폼이 내재되어 99.9% 항균이 가능해요~"
text = re.sub('[^가-힣a-z.]', ' ', text)
ATE_ABSA(text)

tokens: ['고밀도', '폼', '소재', '라', '.', '.', '빈틈', '이', '없', '고', '커버', '만', '관리', '하', '면', '되', '고', '항균', '폼', '이', '내재', '되', '어', '.', '항균', '이', '가능', '해요']
ATE: []


In [79]:
text = """나는 사람이 좋아"""
ATE_ABSA(text)

tokens: ['나', '는', '사람', '이', '좋', '아']
ATE: []


# Cyberpunk 2077 - Xbox One

https://www.amazon.com/-/zh_TW/Cyberpunk-2077-Xbox-One/product-reviews/B07DJW4WZC/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2

In [80]:
text = "Spent 5 hours downloading updates."
ATE_ABSA(text)

tokens: ['Spent', '5', 'hours', 'downloading', 'updates', '.']
ATE: []


In [81]:
text = "Install is buggy, so after downloading a day one patch that's nearly 3 times the size of the game, it glitched on the CDs and had to reinstall the game from scratch."
ATE_ABSA(text)

tokens: ['Install', 'is', 'buggy', ',', 'so', 'after', 'downloading', 'a', 'day', 'one', 'patch', 'that', "'", 's', 'nearly', '3', 'times', 'the', 'size', 'of', 'the', 'game', ',', 'it', 'glitched', 'on', 'the', 'CDs', 'and', 'had', 'to', 'reinstall', 'the', 'game', 'from', 'scratch', '.']
ATE: []


In [82]:
text = "Cyberpunk 2077 freezes constantly, frame rates are terrible, and it's extremely frustrating to try to play."
ATE_ABSA(text)

tokens: ['Cyberpunk', '2077', 'freezes', 'constantly', ',', 'frame', 'rates', 'are', 'terrible', ',', 'and', 'it', "'", 's', 'extremely', 'frustrating', 'to', 'try', 'to', 'play', '.']
ATE: []


In [83]:
text = "Cyberpunk 2077 is completely unplayable on xbox one. They should have never released this for current gen."
ATE_ABSA(text)

tokens: ['Cyberpunk', '2077', 'is', 'completely', 'unplayable', 'on', 'xbox', 'one', '.', 'They', 'should', 'have', 'never', 'released', 'this', 'for', 'current', 'gen', '.']
ATE: []


In [84]:
text = "It’s just a cash grab, the game crashes constantly, runs at like 20 fps, half the environment and characters only load when you’re three feet away from them. Unless you’re in a small space the game looks awful. The worst game i’ve ever played in years visually. It looks worse than later xbox 360 games."
ATE_ABSA(text)

tokens: ['It', '’', 's', 'just', 'a', 'cash', 'grab', ',', 'the', 'game', 'crashes', 'constantly', ',', 'runs', 'at', 'like', '20', 'fps', ',', 'half', 'the', 'environment', 'and', 'characters', 'only', 'load', 'when', 'you', '’', 're', 'three', 'feet', 'away', 'from', 'them', '.', 'Unless', 'you', '’', 're', 'in', 'a', 'small', 'space', 'the', 'game', 'looks', 'awful', '.', 'The', 'worst', 'game', 'i', '’', 've', 'ever', 'played', 'in', 'years', 'visually', '.', 'It', 'looks', 'worse', 'than', 'later', 'xbox', '360', 'games', '.']
ATE: []


In [85]:
text = "CD Projekt Red should have just abandoned the current gen consoles instead of cheating people out of their money."
ATE_ABSA(text)

tokens: ['CD', 'Projekt', 'Red', 'should', 'have', 'just', 'abandoned', 'the', 'current', 'gen', 'consoles', 'instead', 'of', 'cheating', 'people', 'out', 'of', 'their', 'money', '.']
ATE: []
